---
  title: "RestFull 05: API Client"
  description: "REST Clients with Jakarta RESTful Web Services."
  categories: 
    - Java
    - Lecture
    - RestFull
  provide_notes: true
  provide_slides: false
  jupyter: java-lts
  echo: true
  output: true
---

{{< embed ./quarto-utils/_version.qmd >}}

La classe `fr.univtln.bruno.samples.jaxrs.client.BiblioClient` présente un client qui utilise l'API fluent cliente pour construire des requêtes REST en Java.

In [1]:
//| output: false
//| echo: false

%printWithName false
    
// UPDATE SAMPLE SOURCE CODE

String script="""
GITHUB_REPO=ebpro/sample-jaxrs
GITHUB_URL=https://github.com/${GITHUB_REPO}
BRANCH=develop
SRC_DIR=/home/jovyan/work/src/github/${GITHUB_REPO}
gitpuller ${GITHUB_URL} ${BRANCH} ${SRC_DIR}
cd ${SRC_DIR}
mvn --quiet clean package
""";
IJava.getKernelInstance().getMagics().applyCellMagic("shell",List.of(""),script);  

$ git fetch


$ git reset --mixed


$ git -c user.email=nbgitpuller@nbgitpuller.link -c user.name=nbgitpuller merge -Xours origin/develop


Already up to date.


0

In [2]:
//| output: false
//| echo: false
%jars "/home/jovyan/work/src/github/ebpro/sample-jaxrs/target/sample-jaxrs-*-withdependencies.jar"; 
import org.glassfish.grizzly.http.server.HttpServer;
import fr.univtln.bruno.samples.jaxrs.server.BiblioServer;
HttpServer httpServer = BiblioServer.startServer();
httpServer.toString();

org.glassfish.grizzly.http.server.HttpServer@650cefb6

In [3]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/client/BiblioClient.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcClassByName",List.of("BiblioClient"),script);
return null;

```Java
@Log
public class BiblioClient {
    public static void main(String[] args) {
       try (
        // create the rest client
        Client client = ClientBuilder.newClient()) {
           WebTarget webResource = client.target("http://localhost:9998/mylibrary");

           //Send a put with a String as response
           String responseInitAsString = webResource.path("library/init")
                   .request().put(Entity.entity("", MediaType.TEXT_PLAIN), String.class);
           log.info(responseInitAsString);

           //Send a get and parse the response as a String
           String responseAuteursAsJsonString = webResource.path("authors")
                   .request().get(String.class);
           log.info(responseAuteursAsJsonString);

           //Idem but the result is deserialised to an instance of Auteur
           Library.Author author = webResource.path("authors/1")
                   .request()
                   .get(Library.Author.class);
           log.info(author.toString());

           //Log in to get the token with basci authentication
           String email = "john.doe@nowhere.com";
           String passwd = "admin";
           String token = webResource.path("setup/login")
                   .request()
                   .accept(MediaType.TEXT_PLAIN)
                   .header("Authorization", "Basic " + java.util.Base64.getEncoder().encodeToString((email + ":" + passwd).getBytes()))
                   .get(String.class);
           if (!token.isBlank()) {
               log.info("token received.");
               //We access a JWT protected URL with the token
               String result = webResource.path("setup/secured")
                       .request()
                       .header("Authorization", "Bearer " + token)
                       .get(String.class);

               log.info(result);
           }
       }

    }
}
```

In [4]:
%%shell 
cd /home/jovyan/work/src/github/ebpro/sample-jaxrs
mvn  -quiet -Dmain.class="fr.univtln.bruno.samples.jaxrs.client.BiblioClient" exec:java

Oct 03, 2024 1:07:39 PM fr.univtln.bruno.samples.jaxrs.client.BiblioClient main


INFO: 2


Oct 03, 2024 1:07:39 PM fr.univtln.bruno.samples.jaxrs.client.BiblioClient main


INFO: [{"id":1,"name":"Martin","firstname":"Alfred","books":[1,2]},{"id":2,"name":"Durand","firstname":"Marie","books":[2,3,4]}]


Oct 03, 2024 1:07:39 PM fr.univtln.bruno.samples.jaxrs.client.BiblioClient main


INFO: Library.Author(id=1, name=Martin, firstname=Alfred, biography=null)


Oct 03, 2024 1:07:39 PM fr.univtln.bruno.samples.jaxrs.client.BiblioClient main


INFO: token received.


Oct 03, 2024 1:07:39 PM fr.univtln.bruno.samples.jaxrs.client.BiblioClient main


INFO: Access with JWT ok for Doe, John <john.doe@nowhere.com>


In [5]:
//| output: false
//| echo: false
httpServer.stop();